## In this notebook, we plot various candle prices, and the moving average

In [60]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [61]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16, 64]

df = pd.read_pickle(utils.get_hist_data_filename(pair, granularity))
#df.head()
#df.describe()
#df.info()
#df.columns

In [62]:
# convert object(strings) types to integer types for data manipulation
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]

df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [63]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [76]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    
df_ma.dropna(inplace=True)

# reset index
df_ma.reset_index(drop=True, inplace=True)

In [77]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2024-06-19T17:00:00.000000000Z,1.07472,1.07474,1.07440,1.07446,1.074224,1.072956,0.001268,0.001310,False
1,2024-06-19T18:00:00.000000000Z,1.07443,1.07446,1.07425,1.07432,1.074251,1.073017,0.001234,0.001268,False
2,2024-06-19T19:00:00.000000000Z,1.07434,1.07458,1.07428,1.07450,1.074296,1.073081,0.001215,0.001234,False
3,2024-06-19T20:00:00.000000000Z,1.07450,1.07454,1.07433,1.07436,1.074344,1.073143,0.001202,0.001215,False
4,2024-06-19T21:00:00.000000000Z,1.07440,1.07469,1.07437,1.07451,1.074391,1.073214,0.001177,0.001202,False


In [78]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [79]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [81]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [82]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [83]:
df_trades = df_ma[df_ma.IS_TRADE == True].copy()

In [84]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
18,2024-06-20T11:00:00.000000000Z,1.07288,1.07322,1.07130,1.07150,1.073616,1.073628,-0.000012,0.000148,True
69,2024-06-24T14:00:00.000000000Z,1.07426,1.07465,1.07272,1.07301,1.071071,1.071047,0.000024,-0.000226,True
103,2024-06-26T00:00:00.000000000Z,1.07091,1.07126,1.07062,1.07115,1.071136,1.071292,-0.000156,0.000019,True
146,2024-06-27T19:00:00.000000000Z,1.07060,1.07074,1.07014,1.07030,1.070226,1.070168,0.000058,-0.000074,True
328,2024-07-09T09:00:00.000000000Z,1.08224,1.08260,1.08188,1.08227,1.082556,1.082590,-0.000035,0.000041,True


In [85]:
# plot last 100 candles
df_plot = df_ma.iloc[:37].copy()

In [86]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, 
    open=df_plot.mid_o, 
    high=df_plot.mid_h, 
    low=df_plot.mid_l, 
    close=df_plot.mid_c,
    line=dict(width=1),
    opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
))
fig.update_layout(
    width=1000, height=400,
    font=dict(size=10, color="#e1e1e1"),
    margin=dict(l=10, r=10, b=10, t=10),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e"
)
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True, fixedrange=True, #rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()